# 개요
* 참여중인 데이터 엔지니어링 스터디에서 배우는 내용 정리
  * 데이터 수집, 정제 : pyspark, airflow
  * 저장 : elasticsearch
  * 시각화 : kibana

* 최종과제 : 배운 Pyspark, Airflow, Elasticsearch, Kibana로 데이터 파이프라인 만들어보기

* 만들다보니 시각화에 대한 부분을 좀 더 해보고 싶어, 과제2를 추가로 진행함
  * 과제 1 : gharchive 데이터파이프라인 + 대시보드 약간
  * 과제 2 : 4차산업 핵심광물(Un comtrade) 데이터파이프라인 + 대시보드
  * git repo (과제 1, 2 모두 여기에 보관)
    * [https://github.com/KR9268/metacode_de-2024](https://github.com/KR9268/metacode_de-2024)

* 발표 후 피드백 받은 내용을 하단에 정리

# 과제 피드백 정리

## 과제1 피드백
  * Retry 주기는 길지 않게 설정하는 것이 좋다
    * 데이터가 1시간 단위로 업데이트되니 부재일 것을 우려하여 2시간 단위로 Retry한 것으로 보임
    * 하지만 발표에서도 그랬듯이 실제로는 데이터가 없어서 문제가 발생하는게 많지 않음
    * 문제해결을 빠르게 하는 것이 중요하므로, 문제사유가 어떤 것이든 1~2시간은 길고 5~10분이 좋을 듯 하다


## 과제2 피드백
  * 데이터가 수시로 업데이트된다면, 이처럼 업데이트된 것만 현황을 확인하고 가져오는 것이 필수적이고 좋은 전략
    * API형태가 아니라 Kafka 등도 업데이트 된 것만 가져오는 기능을 제공
  * 적절한 제한량 문제
    * 제한이 있으므로 분산처리를 안한다했으나, 제한량이 있을 때 유용한 것이 분산처리이기도 함
    * IP기준으로 사용량을 체크한다면, 여러 노드의 IP대역 등을 분리하는 방안도 있음
      * 현재는 Local환경에서 실습하므로 쓸 수는 없는 방법
      * 크롤링 등은 IP대역으로 체크하므로, 여러 노드에 대역을 할당해 하는 것이 가장 빠름
    * API Key라면 Key를 여러개 받아 관리하는 방법도 있음
  * API로 데이터를 호출했다면, Raw데이터를 보존
    * 복구가 불가능한 상황에 대한 대비
    * 문제 상황에 대한 원인 파악 등 사용(API호출시기에 따라 달라져있으므로 확인이 불가할 수 있음)
      * 제공중이던 학습 데이터가 갑자기 튀어 AI학습에 문제생겼을 때, 확인 등을 위해 사용
  * 생성한 Unique ID의 규칙
    * 과제 진행한 것처럼, 기간이나 카테고리 등으로 들어가는게 맞긴 함(기간+HSCODE+수출국+수입국+수출/수입 여부)
    * 성능개선이 필요하다면, 데이터 쏠림을 막을 수 있는 방법을 고민해야함
      * 과제의 Index로는 특정 수출국이 많다면 데이터 쏠림(Skew)이 발생할 수 있음
      * ID를 한번 해싱(앞에 랜덤해싱을 붙인다던가)하면 분산이 잘 될 수 있음
      * 이러한 일정한 규칙인 Sort값을 앞에 붙여 해싱하면 더 다양하게 해시값이 생성되고 고르게 분산됨
      * 기간, hscode 등 다 알고있는 값을 일방향해싱하면 되므로 만들기 쉽고 분산이 잘될 것임

## 기타 과제 중 궁금했던 점 질의응답
  * Local환경 실습을 위해 며칠 컴퓨터를 켜두었는데 느려지기도 하고 메모리(Ram)점유율이 올라갔는데, 가비지콜렉션(GC)를 따로 적용하면 되는 문제인지?
    * Spark도 GC를 못해서 죽는 경우가 있고(연산량이 크다던가 이유로), 강제로 GC를 호출하는 방법도 있음
      * map이나 For each 등 너무 큰 연산이 있을 때 루프마다 GC를 호출하는 등 방법
    * 문제의 원인은 Airflow일 확률이 높아보임
      * Airflow는 Log rotate를 직접 작성해주어야 함
        * Log rotate는 간단히 말해 주기적으로 로그 등을 삭제해주는 것(가장 최근 것을 링크걸고 나머지 삭제 등 진행)
      * (Airflow가 아직 덜 성숙한 플랫폼이라는 얘기를 했었는데)Dag추가나 Crontab으로 Log rotate구현이 필요해 보임
      * 중간 값 확인 등을 위해 dataframe의 show 등이 많아 로그가 많이 쌓인 것으로 추정